In [ ]:
import xml.etree.ElementTree as ElementTree

class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    def __init__(self, parent_element):
        if parent_element.items():
            self.updateShim( dict(parent_element.items()) )
        for element in parent_element:
            if len(element):
                aDict = XmlDictConfig(element)
            #   if element.items():
            #   aDict.updateShim(dict(element.items()))
                self.updateShim({element.tag: aDict})
            elif element.items():    # items() is specialy for attribtes
                elementattrib= element.items()
                if element.text:           
                    elementattrib.append((element.tag,element.text ))     # add tag:text if there exist
                self.updateShim({element.tag: dict(elementattrib)})
            else:
                self.updateShim({element.tag: element.text})

    def updateShim (self, aDict ):
        for key in aDict.keys():   # keys() includes tag and attributes
            if key in self:
                value = self.pop(key)
                if type(value) is not list:
                    listOfDicts = []
                    listOfDicts.append(value)
                    listOfDicts.append(aDict[key])
                    self.update({key: listOfDicts})
                else:
                    value.append(aDict[key])
                    self.update({key: value})
            else:
                self.update({key:aDict[key]})



In [ ]:
!pip install xmltodict

In [ ]:
import xml.etree.ElementTree as ElementTree

import xmltodict
import pprint
import json
import pandas as pd
import numpy as np
from datetime import date

#file input
tree = ElementTree.parse('./raw/BUILT.30.experienced_plans.xml')
root = tree.getroot()
xmldict = XmlDictConfig(root)



In [ ]:
#initiate activity related list object 
id_data_act = []
activity_data = []
start_time = []
end_time = []
for type_tag in root.findall('person'):
    for act in type_tag.findall('plan/activity'):
        id_data_act.append(type_tag.get('id')) #getting id
        activity_data.append(act.get('type'))  #getting work, home 
        start_time.append(act.get('start_time'))
        end_time.append(act.get('end_time'))


In [ ]:
#initiate mode related list object 

id_data_mode = []
mode_data = []
dist_data = []
travelTime_data = []

for type_tag in root.findall('person'):
    if len(type_tag.findall('plan/leg')) == 0:
        id_data_mode.append(type_tag.get('id')) 
        mode_data.append('na')
        dist_data.append('na')
        travelTime_data.append('na')
  
    for i ,mode in enumerate(type_tag.findall('plan/leg')):
        id_data_mode.append(type_tag.get('id')) 
        mode_data.append(mode.get('mode'))
        
        route = mode.find('route')
        dist_data.append(route.get('distance'))
        travelTime_data.append(route.get('trav_time'))

        if i == len(type_tag.findall('plan/leg'))-1:
            id_data_mode.append(type_tag.get('id')) 
            mode_data.append('N/A')
            dist_data.append('N/A')
            travelTime_data.append('N/A')

actList =  list(zip(id_data_act, activity_data, start_time, end_time))
modeList = list(zip(mode_data, dist_data, travelTime_data))
df_act = pd.DataFrame(actList, columns=['id','activity','start_time','end_time'])
df_mode = pd.DataFrame(modeList, columns=['mode', 'dist','travel_time'])
df_merged = pd.concat([df_act, df_mode],axis = 1)

In [ ]:
#Calculate the total travel time between
id_list = []
start_time_sec = []
end_time_sec = []
travel_time_sec =[]
departure_time = []



for i in range(0,len(df_merged)):#
    id_list.append( df_merged.iloc[i]['id'])

    if( df_merged.iloc[i]['start_time']!=None):
        ls = df_merged.iloc[i]['start_time'].split(':')
        start_time_sec.append(int(ls[0])*3600+int(ls[1])*60+int(ls[2])*1)
    else:
        start_time_sec.append(0)
        
        
    if( df_merged.iloc[i]['end_time']!=None):
        ls = df_merged.iloc[i]['end_time'].split(':')
        end_time_sec.append(int(ls[0])*3600+int(ls[1])*60+int(ls[2])*1)
        departure_time.append(int(ls[0]))

    else:
        end_time_sec.append(0)
        departure_time.append(0)


timeList =  list(zip(id_list, start_time_sec, end_time_sec, departure_time))
df_time = pd.DataFrame(timeList, columns=['id2','start_time_sec','end_time_sec','dept_time'])

print("1 is done")
     
        
    

In [ ]:

time_diff = []
time_diff.append(np.nan)

for i in range(0,len(df_time)-1):
    
    if(df_time.iloc[i]['id2']==df_time.iloc[i+1]['id2']):
        time_diff.append(df_time.iloc[i+1]['start_time_sec'] - df_time.iloc[i]['end_time_sec'] )
    else:
        time_diff.append(np.nan)


        
df_time['travel_time']=time_diff

print("2 is done")



In [ ]:
#Export files
df_final =  pd.concat([df_merged, df_time], axis=1)
df_final.head()
df_final.to_excel("DataFrame_with_TT.xlsx")
print("exported the file 1")

In [ ]:
df_agent_travelTime=df_final[['travel_time','mode']].groupby(df_final['id']).sum()
df_agent_travelTime.to_excel("TravelTT_only.xlsx")
print("exported the file 2")

